# CARS.csv Data Demonstration

In [ ]:
#Import Libraries
import pandas as pd
import sys
from os import listdir
import re
from collections import Counter
import matplotlib.pyplot as plt
from googlesearch import search
from matplotlib.figure import Figure
import numpy as np
import webbrowser

### Upload CSV File

In [ ]:
rawdf = pd.read_csv('cars.csv')

## Cleaning and Exploratory Data Analysis

### Summary

In [ ]:
print("rows:",df.shape[0])
print("field:",df.shape[1])
rawdf.head(10)

### Count Nulls

In [ ]:
rawdf.isna().sum()

### Are Nulls Clustered in the same rows?

In [ ]:
torquenull_df = rawdf[rawdf.torque.isnull()]
print("Rows w/ Torque Nulls w/o: Max Power Nulls:",torquenull_df[torquenull_df.max_power.notnull()].shape[0])
print("Rows w/ Torque Nulls w/o: Engine Nulls:",torquenull_df[torquenull_df.engine.notnull()].shape[0])
print("Rows w/ Torque Nulls w/o: Seats Nulls:",torquenull_df[torquenull_df.seats.notnull()].shape[0])
print("Rows w/ Torque Nulls w/o: Mileage Nulls:",torquenull_df[torquenull_df.mileage.notnull()].shape[0])

### Remove Nulls

In [ ]:
df1 = rawdf.dropna()
df1.isna().sum()

In [ ]:
print("rows:",df1.shape[0])

### Check Field Data Types

In [ ]:
df1.dtypes

### Convert Seats from Float into Int

In [ ]:
df2 = df1
#df2.loc[df2['seats']] = df2.loc[df2['seats']].astype('Int64')
df2 = df2.astype({'seats': 'int64'})
df1.dtypes

### Convert Mileage from String into Number

#### Test Mileage field - be suspicious of Units of Measure (here, "kmpl")

In [ ]:
milecheck_df = df2
mileage = []  

while True:
    try:
        for values in milecheck_df['mileage']:
            mileage.append(re.search(r"([0-9.]+) kmpl", values).group(1))
        break
    except:
        print("Regex doesn't catch entire field.")
        print(values)
        break

#### Convert "km/kg" to "kmpl", convert all results from String to Float, and rename header to include UoM

In [ ]:
# Evidently, 1.4 km/kg = 1 kmpl according to an internet search
# https://www.kaggle.com/code/rushikeshhiray/converting-km-kg-to-kmpl/notebook

milecheck_df = df2
mileage = []  

while True:
    try:
        for values in milecheck_df['mileage']:
            raw_mileage = float(re.search(r"([0-9.]+) (kmpl|km\/kg)", values).group(1))
            if re.search(r"([0-9.]+) (kmpl|km\/kg)", values).group(2) == "km/kg":
                raw_mileage /= 1.4
            mileage.append(raw_mileage)  
        break
    except:
        print("Regex doesn't catch entire field.")
        print(values)
        break

df3 = df2
df3['mileage'] = mileage
df3 = df3.rename(columns={"mileage": "mileage (kmpl)"})
df3.head()

### Convert Engine from String into Number

#### Test Engine field - be suspicious of Units of Measure (here, "CC")

In [ ]:
enginecheck_df = df3
engine = []  

while True:
    try:
        for values in enginecheck_df['engine']:
            engine.append(re.search(r"([0-9.]+) CC", values).group(1))
        break
    except:
        print("Regex doesn't catch entire field.")
        print(values)
        break

#### Single UoM present, convert all results from String to Float, and rename header to include UoM

In [ ]:
df4 = df3
engine = [float(value) for value in engine]
df4['engine'] = engine
df4 = df4.rename(columns={"engine": "engine (CC)"})
df4.head()

### Convert Max Power from String into Number

#### Test Max Power field - be suspicious of Units of Measure (here, "bhp")

In [ ]:
powercheck_df = df4
power = []  

while True:
    try:
        for values in powercheck_df['max_power']:
            power.append(re.search(r"([0-9.]+) bhp", values).group(1))
        break
    except:
        print("Regex doesn't catch entire field.")
        print(values)
        break

#### Single UoM present, convert all results from String to Float, and rename header to include UoM

In [ ]:
df5 = df4
power = [float(value) for value in power]
df5['max_power'] = power
df5 = df5.rename(columns={"max_power": "max_power (bhp)"})
df5.head()

### Parse Torque

#### Inspect sample to find format variants

In [ ]:
for x in df5['torque'][:30]:
    print(x)

#### Test regex to ensure it catches all formats and convert KGMs to NMs

In [ ]:
# 1 kilogram-meter (kgm) = 9.80665 Newtom-meters (nm)

torque_df = df5
torque,rpm = [],[]

for values in torque_df['torque']:
    if re.search(r"([0-9.]+)[ ]*[NnMm]+[ ]*[@]*[at]*[ ]+([\-0-9.~]+)[ rpm]*", values) is not None:
        torque.append(float(re.search(r"([0-9.]+)[ ]*[NnMm]+[ ]*[@]*[at]*[ ]+([\-0-9.~]+)[ rpm]*", values).group(1)))
        rpm.append(re.search(r"([0-9.]+)[ ]*[NnMm]+[ ]*[@]*[at]*[ ]+([\-0-9.~]+)[ rpm]*", values).group(2))
    elif re.search(r"([0-9.]+)[ ]*[KkGgMm]*[ ]*[@]*[at]*[ ]+([\-0-9.~,]+)[ RrPpMm]*",values) is not None:
        torval = re.search(r"([0-9.]+)[ ]*[KkGgMm]*[ ]*[@]*[at]*[ ]+([\-0-9.~,]+)[ RrPpMm]*", values).group(1)
        multiple = 9.80665 * float(torval) 
        torque.append(multiple)
        newrpm = re.search(r"([0-9.]+)[ ]*[KkGgMm]*[ ]*[@]*[at]*[ ]+([\-0-9.~,]+)[ RrPpMm]*",values).group(2)
        rpm.append(newrpm)
        #print(values,"org torque:",torval,"math torque:",multiple,"rpm:",newrpm)
    elif re.search(r"([0-9.]+)Nm\([0-9.]+kgm\)@ ([0-9.]+)rpm", values) is not None:
        torque.append(float(re.search(r"([0-9.]+)Nm\([0-9.]+kgm\)@ ([0-9.]+)rpm", values).group(1)))
        rpm.append(re.search(r"([0-9.]+)Nm\([0-9.]+kgm\)@ ([0-9.]+)rpm", values).group(2))
    elif re.search(r"([0-9.]+) Nm \/([0-9.]+) rpm", values) is not None:
        torque.append(float(re.search(r"([0-9.]+) Nm \/([0-9.]+) rpm", values).group(1)))
        rpm.append(re.search(r"([0-9.]+) Nm \/([0-9.]+) rpm", values).group(2))
    elif re.search(r"([0-9.]+)\([0-9.]+\)@ ([0-9.]+)", values) is not None:
        torque.append(float(re.search(r"([0-9.]+)\([0-9.]+\)@ ([0-9.]+)", values).group(1)))
        rpm.append(re.search(r"([0-9.]+)\([0-9.]+\)@ ([0-9.]+)", values).group(2)) 
    elif re.search(r"([0-9.]+) \/ ([0-9.]+)", values) is not None:
        torque.append(float(re.search(r"([0-9.]+) \/ ([0-9.]+)", values).group(1)))
        rpm.append(re.search(r"([0-9.]+) \/ ([0-9.]+)", values).group(2))
    elif re.search(r"([0-9.]+)Nm", values) is not None:
        torque.append(float(re.search(r"([0-9.]+)Nm", values).group(1)))
        rpm.append("Unknown")            
    else:
        print(values)
    torque = [round(value) for value in torque]

In [ ]:
df6 = df5
df6['torque'] = torque
df6['rpm'] = rpm
df6 = df6.rename(columns={"torque": "torque (nm)"})
df6.head()

### Replace Name Field with Make and Model Fields

In [ ]:
make,model = [],[]    
for values in df6['name']:
    make.append(re.search(r"([a-zA-Z0-9\-]*)[' ']([a-zA-Z0-9\-\.\(\) ]*)", values).group(1))
    model.append(re.search(r"([a-zA-Z0-9\-]*)[' ']([a-zA-Z0-9\-\.\(\) ]*)", values).group(2))
df7 = df6
df7['make'],df7['model'] = make,model

df7 = df7.drop(['name'],axis=1)

cols = df7.columns.tolist()
cols = cols[-2:] + cols[:-2]
df7 = df7[cols]
df7.head(5)

### Count of Unique Values Per Field

In [ ]:
#End of Cleaning; final df
df = df7

print(df.nunique())

### List Unique Values for Fields with <100 Uniques (Ordered by Counts)

In [ ]:
for (colname,coldata) in df.iteritems():
    if len(coldata.unique()) < 100:
        print(colname.upper()+"  /  COUNT")
        print(df[colname].value_counts())            
        print("\n","-"*100,"\n")

### Assorted Plots

#### Make - Bar Plot

In [ ]:
colname = 'make'
df[colname].value_counts().plot(kind='bar')
plt.style.use('seaborn-muted')
plt.rcParams['figure.figsize'] = [22, 8]
plt.rc('font', size=12) 
plt.xticks(rotation=65)
plt.title(label = colname.upper(),fontdict={'fontsize': 24})
plt.show()

#### Seats - Bar Plot

In [ ]:
colname = 'seats'
df[colname].value_counts().sort_index().plot(kind='bar')
plt.rcParams['figure.figsize'] = [22, 10]
plt.xticks(rotation=0)
plt.rc('font', size=16) 
plt.title(label = colname.upper(),fontdict={'fontsize': 24})
plt.show()

#### Selling Price - Histogram

In [ ]:
colname = 'selling_price'
df.hist(column= colname,bins=100)
plt.rcParams['figure.figsize'] = [22, 10]
plt.title(label = colname.upper(),fontdict={'fontsize': 24})
plt.xticks(rotation=45)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,max(df['selling_price'])))
plt.show()

#### KM Driven - Histogram

In [ ]:
colname = 'km_driven'
df.hist(column= colname,bins=100)
plt.rcParams['figure.figsize'] = [22, 10]
plt.title(label = colname.upper(),fontdict={'fontsize': 24})
plt.xticks(rotation=45)
plt.show()

#### Year - Histogram

In [ ]:
colname = 'year'
df.hist(column= colname,bins=40)
plt.rcParams['figure.figsize'] = [22, 10]
plt.title(label = colname.upper(),fontdict={'fontsize': 24})
plt.show()

#vals = df[df['server'].isin([1,3])].groupby(['court'])['speed'].mean().to_frame()

### Averages of numeric fields by Make

In [ ]:
averages_df = df.groupby(['make']).mean().round(2)
averages_df

### Statistical Summary for Numeric Fields

In [ ]:
df.describe().apply(lambda s: s.apply('{0:.0f}'.format))

## Seating Capacity Analysis

### List of Cars with Exceptional Seat Capacity (cars with more than eight seats)

In [ ]:
pd.set_option('display.max_rows', None)
hi_seats = df6[df6["seats"] > 8].filter(items=['name', 'seats']).sort_values('seats', ascending=False)
hi_seats = hi_seats.drop_duplicates()

hi_seats

### Google searches for largest seating vehicles (Top 5 cars, top 3 Google results)

In [ ]:
for car in hi_seats['name'][:5]:
    print(car.upper())
    for j in search(car, tld="co.in", stop=3):
        print(j)
    print("\n")

### Open first browser tabs for top three large-seat cars

In [ ]:
for car in hi_seats['name'][:3]:
    print(car.upper())
    for site in search(car, tld="co.in", stop=1):
        webbrowser.open(site)

## Correlation

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

correlation_df = df.filter(items=
   ['make', 'year','selling_price','km_driven','mileage (kmpl)','engine (CC)','seats'])

correlation_df.head()

In [ ]:
correlations = correlation_df.corr()

correlations

In [ ]:
%matplotlib inline
import seaborn as sns
from matplotlib.pyplot import figure
figure(figsize=(16, 12), dpi=80)
#plt.rcParams['figure.figsize'] = [1, 1]
plt.yticks(rotation=45)
#plt.xticks(rotation = 45)

sns.heatmap(correlations)
plt.title(label = 'CORRELATIONS',fontdict={'fontsize': 16})

plt.show()